<a href="https://colab.research.google.com/github/Zarika14/Analysis_Project/blob/main/Twitterr_sentiment_analysis/Twitter_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
#configuring the path of kaggle.json file
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


Now we will Download the Dataset from kaggle

In [25]:
#API to fetch the dataset from Kaggle
!kaggle datasets download -d jp797498e/twitter-entity-sentiment-analysis

Dataset URL: https://www.kaggle.com/datasets/jp797498e/twitter-entity-sentiment-analysis
License(s): CC0-1.0
twitter-entity-sentiment-analysis.zip: Skipping, found more recently modified local copy (use --force to force download)


In [26]:
#extracting the zip file
from zipfile import ZipFile
dataset='/content/twitter-entity-sentiment-analysis.zip'

with ZipFile(dataset, 'r') as zip:
  zip.extractall()


Now we need to Import the Dependencies for our Project.

In [27]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [28]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [29]:
#print stopwords in English
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [30]:
#loading data from csv file to panda dataframe
twitter_data= pd.read_csv('/content/twitter_training.csv', encoding ='ISO-8859-1')

In [31]:
# Number of rows and columns
twitter_data.shape

(74681, 4)

In [32]:
#printing first 5 rows of dataframe
twitter_data.head()

,2401,Borderlands,Positive,"im getting on borderlands and i will murder you all ,"
0,2401,Borderlands,Positive,I am coming to the borders and I will kill you...
1,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...
2,2401,Borderlands,Positive,im coming on borderlands and i will murder you...
3,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...
4,2401,Borderlands,Positive,im getting into borderlands and i can murder y...


In [33]:
#naming column dataset
column_names= ['Tweet_id','Entity','The polarity of the tweet','tweet_content']
twitter_data= pd.read_csv('/content/twitter_training.csv',names= column_names, encoding ='ISO-8859-1')

In [34]:
twitter_data.shape

(74682, 4)

In [35]:
twitter_data.head()

,Tweet_id,Entity,The polarity of the tweet,tweet_content
0,2401,Borderlands,Positive,im getting on borderlands and i will murder yo...
1,2401,Borderlands,Positive,I am coming to the borders and I will kill you...
2,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...
3,2401,Borderlands,Positive,im coming on borderlands and i will murder you...
4,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...


In [36]:
#Check for missing values in the dataset
twitter_data.isnull().sum()

Tweet_id                       0
Entity                         0
The polarity of the tweet      0
tweet_content                686
dtype: int64

In [37]:
twitter_data[twitter_data['tweet_content'].isna() == True]

,Tweet_id,Entity,The polarity of the tweet,tweet_content
61,2411,Borderlands,Neutral,NaN
553,2496,Borderlands,Neutral,NaN
589,2503,Borderlands,Neutral,NaN
745,2532,Borderlands,Positive,NaN
1105,2595,Borderlands,Positive,NaN
...,...,...,...,...
73972,9073,Nvidia,Positive,NaN
73973,9073,Nvidia,Positive,NaN
74421,9154,Nvidia,Positive,NaN
74422,9154,Nvidia,Positive,NaN


In [38]:
# dropping the rows as it has no review written in body
twitter_data.dropna(inplace=True)

In [39]:
twitter_data.isnull().sum()

Tweet_id                     0
Entity                       0
The polarity of the tweet    0
tweet_content                0
dtype: int64

In [40]:
#check for target distribution
twitter_data['The polarity of the tweet'].value_counts()

The polarity of the tweet
Negative      22358
Positive      20655
Neutral       18108
Irrelevant    12875
Name: count, dtype: int64

In [41]:
# dropping the irrelevant columns
twitter_data.drop(twitter_data[twitter_data['The polarity of the tweet'] == 'Irrelevant'].index, inplace=True)

In [42]:
print(twitter_data['The polarity of the tweet'].value_counts())

The polarity of the tweet
Negative    22358
Positive    20655
Neutral     18108
Name: count, dtype: int64


In [43]:
twitter_data.replace({'The polarity of the tweet': {'Positive': 1, 'Negative': -1, 'Neutral': 0}}, inplace=True)

In [44]:
twitter_data['The polarity of the tweet'].value_counts()

The polarity of the tweet
-1    22358
 1    20655
 0    18108
Name: count, dtype: int64

-1--> Negative 0--> Neutral 1--> Positive

Stemming: The process of reducing a word to its root word eg. act is root word for actor, acting.

In [45]:
port_stem = PorterStemmer()

In [46]:
def stemming(content):

  stemmed_content= re.sub('[^a-zA-Z]',' ',content)
  stemmed_content= stemmed_content.lower()
  stemmed_content= stemmed_content.split()
  stemmed_content= [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
  stemmed_content = ' '.join(stemmed_content)

  return stemmed_content

In [48]:
# Filter rows where 'tweet_content' is NaN
twitter_data['stemmed_content']= twitter_data['tweet_content'].apply(stemming)

In [49]:
twitter_data.head()

,Tweet_id,Entity,The polarity of the tweet,tweet_content,stemmed_content
0,2401,Borderlands,1,im getting on borderlands and i will murder yo...,im get borderland murder
1,2401,Borderlands,1,I am coming to the borders and I will kill you...,come border kill
2,2401,Borderlands,1,im getting on borderlands and i will kill you ...,im get borderland kill
3,2401,Borderlands,1,im coming on borderlands and i will murder you...,im come borderland murder
4,2401,Borderlands,1,im getting on borderlands 2 and i will murder ...,im get borderland murder


In [50]:
print(twitter_data['stemmed_content'])

0                                 im get borderland murder
1                                         come border kill
2                                   im get borderland kill
3                                im come borderland murder
4                                 im get borderland murder
                               ...                        
74677    realiz window partit mac like year behind nvid...
74678    realiz mac window partit year behind nvidia dr...
74679    realiz window partit mac year behind nvidia dr...
74680    realiz window partit mac like year behind nvid...
74681    like window partit mac like year behind driver...
Name: stemmed_content, Length: 61121, dtype: object


In [51]:
print(twitter_data['The polarity of the tweet'])

0        1
1        1
2        1
3        1
4        1
        ..
74677    1
74678    1
74679    1
74680    1
74681    1
Name: The polarity of the tweet, Length: 61121, dtype: int64


In [52]:
#seperating the data and label

X= twitter_data['stemmed_content'].values
Y= twitter_data['The polarity of the tweet'].values

In [53]:
print(X)

['im get borderland murder' 'come border kill' 'im get borderland kill'
 ... 'realiz window partit mac year behind nvidia driver idea notic'
 'realiz window partit mac like year behind nvidia driver car fuck idea ever notic'
 'like window partit mac like year behind driver idea notic']


In [56]:
# Convert variable Y to a list and print it
print(Y.tolist())

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, -1, -1, -1, -1, -1, -1, 1, 1, 1, 1, 1, 1, -1, -1, -1, -1, -1, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, -1, -1, -1, -1, -1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, -1, -1, -1, -1, -1, -1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, -1, -1, -1, -1, -1, -1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, -1, -1, -1, -1, -1, -1, 1, 1, 1, 1, 1, 1, -1, -1, -1, -1, -1, -1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, -1, -1, -1, -1, -1, -1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, -1, -1, -1, -1, -1, -1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

Splitting the data to training data and test data

In [55]:
# Split dataset X and target variable Y into training and testing sets,
# with a test size of 20%, stratify Y, and set random state to 2
X_train, X_test, Y_train, Y_test= train_test_split(X,Y, test_size=0.2, stratify= Y, random_state= 2)

In [57]:
print(X.shape, X_train, X_test.shape)

(61121,) ['unk'
 'bi vaccin bitutwala mean scaremong mysteri deadli flu viru must kept peopl invest money'
 'play lot unrank dota sea almost toxic peopl fine wtv play everyon chill like get first love'
 ...
 'rockstargam never play red dead redempt game last week western thing realli appeal damn know play everi game last game made highest rate'
 'honor part event one bright star one bright star look forward host celebr life gala saturday februari verizon center mankato mn instagram com p b lftcn xy'
 'love battl warcraft'] (12225,)


In [58]:
#feature extraction(converting textual data to numerical data)
vectorizer = TfidfVectorizer()

X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

In [59]:
print(X_train)

  (0, 16714)	1.0
  (1, 10122)	0.18628795478859106
  (1, 7781)	0.26296367468224263
  (1, 11661)	0.15067730331527232
  (1, 8437)	0.2595199357052159
  (1, 10328)	0.21872991688294688
  (1, 17173)	0.25740460763639306
  (1, 5489)	0.32880204407819774
  (1, 3611)	0.2930713161704539
  (1, 10378)	0.27698572199278165
  (1, 13832)	0.3385355451670683
  (1, 9704)	0.19791116786637433
  (1, 1500)	0.3385355451670683
  (1, 16938)	0.20441724565103458
  (1, 1420)	0.33331997578117883
  (2, 9235)	0.16197171187267026
  (2, 5396)	0.1898713415779162
  (2, 6084)	0.1448355237515061
  (2, 9015)	0.14976570081815113
  (2, 2522)	0.2920135435464757
  (2, 4899)	0.2100202542956277
  (2, 17885)	0.3978965033667672
  (2, 5367)	0.2737880549485174
  (2, 16146)	0.2663432255282015
  (2, 382)	0.23088710014455988
  :	:
  (48894, 8953)	0.24577329728230232
  (48894, 10061)	0.24961899444512767
  (48894, 9508)	0.24961899444512767
  (48894, 5220)	0.21720493768341698
  (48894, 5886)	0.24577329728230232
  (48894, 1888)	0.4223770223334

In [60]:
print(X_test)

  (0, 9303)	0.7142014107080916
  (0, 6327)	0.45221831900259374
  (0, 5805)	0.5342423952673919
  (1, 17809)	0.22400383046199057
  (1, 14669)	0.32239999978065825
  (1, 14022)	0.32631371206246307
  (1, 5135)	0.3066564842579779
  (1, 4232)	0.4338294662006692
  (1, 2250)	0.3578986855714024
  (1, 1487)	0.2576723205185797
  (1, 367)	0.5125106695701225
  (2, 18183)	0.40101217482820795
  (2, 15141)	0.6465128002816147
  (2, 12091)	0.39991095462765275
  (2, 6529)	0.5111571804057683
  (3, 14242)	0.30096969539317364
  (3, 14063)	0.23239915252722773
  (3, 13413)	0.3336653868908038
  (3, 11594)	0.23920964993320928
  (3, 11143)	0.23566918005695273
  (3, 9614)	0.3368995088373444
  (3, 9142)	0.31116730242380014
  (3, 8173)	0.2630504236685428
  (3, 7865)	0.2491120070177313
  (3, 6549)	0.21819715493584727
  :	:
  (12222, 7204)	0.2625522612160462
  (12222, 6084)	0.14144536039119815
  (12222, 5574)	0.27893457927230875
  (12222, 4939)	0.26766611591581674
  (12222, 4847)	0.2926220033792756
  (12222, 995)	0.24

Training the Machine Learning Model

Logistic Regression

In [61]:
model= LogisticRegression(max_iter=2000)

In [62]:
model.fit(X_train, Y_train)

LogisticRegression(max_iter=2000)

Model Evaluation

Accuracy Score

In [63]:
#accuracy score on training data
X_train_prediction = model.predict(X_train)
training_data_accuracy= accuracy_score(Y_train, X_train_prediction)

In [64]:
print('Accuracy score on training data: ', training_data_accuracy)

Accuracy score on training data:  0.8655309227748691


Model accuracy= 86.5%

Saving the Trained Model

In [65]:
import pickle

In [66]:
filename= 'trained_model.pkl'
pickle.dump(model, open(filename,'wb'))

**Model Testing**

In [67]:
#loading the saved model
loaded_model = pickle.load(open('/content/trained_model.pkl','rb'))

In [68]:
X_new= X_test[3]
print(Y_test[3])

prediction = model.predict(X_new)
print(prediction)

if (prediction[0]==-1):
  print('Negative tweet')

if(prediction[0]==0):
  print('Neutral Tweet')

if(prediction[0]==1):
  print('Positive Tweet')

-1
[-1]
Negative tweet


In [70]:
X_new= X_test[2]
print(Y_test[2])

prediction = model.predict(X_new)
print(prediction)

if (prediction[0]==-1):
  print('Negative tweet')

if(prediction[0]==0):
  print('Neutral Tweet')

if(prediction[0]==1):
  print('Positive Tweet')

1
[1]
Positive Tweet


In [75]:
X_new= X_test[8]
print(Y_test[8])

prediction = model.predict(X_new)
print(prediction)

if (prediction[0]==-1):
  print('Negative tweet')

if(prediction[0]==0):
  print('Neutral Tweet')

if(prediction[0]==1):
  print('Positive Tweet')

0
[0]
Neutral Tweet


In [69]:
import pickle

# Load the trained model
loaded_model = pickle.load(open('/content/trained_model.pkl', 'rb'))

def predict_sentiment(input_text):
    # Preprocess the input text
    preprocessed_text = stemming(input_text)  # Assuming you have defined the 'stemming' function

    # Vectorize the preprocessed text
    vectorized_text = vectorizer.transform([preprocessed_text])

    # Predict the sentiment using the loaded model
    prediction = loaded_model.predict(vectorized_text)

    # Interpret the prediction
    if prediction[0] == -1:
        return 'Negative review'
    elif prediction[0] == 0:
        return 'Neutral review'
    elif prediction[0] == 1:
        return 'Positive review'

# Example usage
input_review = "very good"
sentiment_prediction = predict_sentiment(input_review)
print("Sentiment prediction:", sentiment_prediction)

Sentiment prediction: Positive review
